# Gesture Recognition


In [6]:
pip install imageio pillow


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os

import imageio.v2 as imageio
from PIL import Image
import datetime

In [3]:
np.random.seed(42)
import random as rn
rn.seed(42)
import tensorflow as tf
from tensorflow.keras import backend as K
tf.random.set_seed(42)

In [4]:
train_data = np.random.permutation(open('D:/Downloads/Project_data (1)/Project_data/train.csv').readlines())
val_data = np.random.permutation(open('D:/Downloads/Project_data (1)/Project_data/val.csv').readlines())
batch_size = 64

In [5]:
import os
import numpy as np
import imageio
from PIL import Image

def data_generator(base_path, data_list, batch_size):
    print(f'Base path = {base_path}, batch size = {batch_size}')
    image_indices = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 27, 28, 29]
    
    while True:
        shuffled_data = np.random.permutation(data_list)
        total_batches = len(shuffled_data) // batch_size
        
        for batch_num in range(total_batches):
            x_batch = np.zeros((batch_size, 18, 84, 84, 3), dtype=np.float32)
            y_batch = np.zeros((batch_size, 5), dtype=np.float32)
            
            for i in range(batch_size):
                folder_name = shuffled_data[batch_num * batch_size + i].split(';')[0]
                image_files = os.listdir(os.path.join(base_path, folder_name))
                
                for j, idx in enumerate(image_indices):
                    img_path = os.path.join(base_path, folder_name, image_files[idx])
                    image = imageio.imread(img_path).astype(np.float32)
                    
                    if image.shape[1] == 160:
                        image = image[:, 20:140, :]  # Crop the image
                    
                    # Ensure the image is in uint8 format for PIL
                    image_uint8 = np.clip(image, 0, 255).astype(np.uint8)
                    image_resized = Image.fromarray(image_uint8).resize((84, 84))
                    image_resized = np.array(image_resized, dtype=np.float32)

                    # Normalize the image
                    x_batch[i, j] = image_resized - np.array([104, 117, 123], dtype=np.float32)

                label_index = int(shuffled_data[batch_num * batch_size + i].strip().split(';')[2])
                y_batch[i, label_index] = 1.0  # Assuming one-hot encoding for labels

            yield x_batch, y_batch

        # Handle remaining data if not a perfect batch
        if len(shuffled_data) % batch_size != 0:
            remaining_data = len(shuffled_data) % batch_size
            x_batch = np.zeros((remaining_data, 18, 84, 84, 3), dtype=np.float32)
            y_batch = np.zeros((remaining_data, 5), dtype=np.float32)
            
            for i in range(remaining_data):
                folder_name = shuffled_data[total_batches * batch_size + i].split(';')[0]
                image_files = os.listdir(os.path.join(base_path, folder_name))
                
                for j, idx in enumerate(image_indices):
                    img_path = os.path.join(base_path, folder_name, image_files[idx])
                    image = imageio.imread(img_path).astype(np.float32)
                    
                    if image.shape[1] == 160:
                        image = image[:, 20:140, :]  # Crop the image
                    
                    # Ensure the image is in uint8 format for PIL
                    image_uint8 = np.clip(image, 0, 255).astype(np.uint8)
                    image_resized = Image.fromarray(image_uint8).resize((84, 84))
                    image_resized = np.array(image_resized, dtype=np.float32)

                    # Normalize the image
                    x_batch[i, j] = image_resized - np.array([104, 117, 123], dtype=np.float32)

                label_index = int(shuffled_data[total_batches * batch_size + i].strip().split(';')[2])
                y_batch[i, label_index] = 1.0  # Assuming one-hot encoding for labels

            yield x_batch, y_batch


In [6]:
# Model training configuration
curr_dt = datetime.datetime.now()
train_path = "D:/Downloads/Project_data (1)/Project_data/train"
val_path = "D:/Downloads/Project_data (1)/Project_data/val"
train_samples = len(train_data)
val_samples = len(val_data)
epochs = 30
print(f'# training samples = {train_samples}')
print(f'# validation samples = {val_samples}')
print(f'# epochs = {epochs}')

# training samples = 663
# validation samples = 100
# epochs = 30


## Model


In [8]:
# Model definition
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dropout, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

model = Sequential()
model.add(Input(shape=(18, 84, 84, 3)))

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))

model.add(Conv3D(128, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(256, (3, 3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [9]:
# Model compilation
sgd_optimizer = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

C:\Users\Venkat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 18, 84, 84, 64)      │           5,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 18, 84, 84, 64)      │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 18, 84, 84, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 9, 42, 84, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 9, 42, 84, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 9, 42, 84, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 9, 42, 84, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 4, 21, 42, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 4, 21, 42, 256)      │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 4, 21, 42, 256)      │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 4, 21, 42, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 2, 10, 21, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 107520)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 107520)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      55,050,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,166,661 (214.26 MB)

 Trainable params: 56,165,765 (214.26 MB)

 Non-trainable params: 896 (3.50 KB)

None


In [10]:
# Generators
train_gen = data_generator(train_path, train_data, batch_size)
val_gen = data_generator(val_path, val_data, batch_size)

In [11]:
# Callbacks
model_dir = f"model_{curr_dt.strftime('%Y%m%d_%H%M%S')}/"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

checkpoint_path = os.path.join(model_dir, 'model-{epoch:02d}-{loss:.4f}-{categorical_accuracy:.4f}-{val_loss:.4f}-{val_categorical_accuracy:.4f}.keras')
checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min', min_delta=1e-4, cooldown=0, min_lr=1e-6)
callbacks = [checkpoint_callback, lr_scheduler]


In [12]:
# Steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

In [14]:
# Model training
epochs = 15
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1,
          callbacks=callbacks, validation_data=val_gen, validation_steps=validation_steps, initial_epoch=0)

C:\Users\Venkat\AppData\Local\Temp\ipykernel_20048\1389588829.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)


Epoch 1/15
 6/10 ━━━━━━━━━━━━━━━━━━━━ 5:17 79s/step - categorical_accuracy: 0.2394 - loss: 13.4528

C:\Users\Venkat\AppData\Local\Temp\ipykernel_20048\1389588829.py:54: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 76s/step - categorical_accuracy: 0.2506 - loss: 12.1619  Base path = D:/Downloads/Project_data (1)/Project_data/val, batch size = 64

Epoch 1: val_loss improved from inf to 5.85020, saving model to model_20241110_111721/model-01-8.8821-0.2805-5.8502-0.2656.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 829s 85s/step - categorical_accuracy: 0.2533 - loss: 11.8638 - val_categorical_accuracy: 0.2656 - val_loss: 5.8502 - learning_rate: 0.0010
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 76s/step - categorical_accuracy: 0.4518 - loss: 1.5342  
Epoch 2: val_loss did not improve from 5.85020
10/10 ━━━━━━━━━━━━━━━━━━━━ 782s 77s/step - categorical_accuracy: 0.4524 - loss: 1.5333 - val_categorical_accuracy: 0.1944 - val_loss: 8.0593 - learning_rate: 0.0010
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 81s/step - categorical_accuracy: 0.4924 - loss: 1.5087  
Epoch 3: val_loss did not improve from 5.85020
10/10 ━━━━━━━━━━━━━━━━━━━━ 887s 83s/step - categorical_accuracy: 0.4961 - loss: 1.4928

In [15]:
# Model evaluation
val_loss, val_accuracy = model.evaluate(val_gen, steps=validation_steps)
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

C:\Users\Venkat\AppData\Local\Temp\ipykernel_20048\1389588829.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)
C:\Users\Venkat\AppData\Local\Temp\ipykernel_20048\1389588829.py:54: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path).astype(np.float32)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - categorical_accuracy: 0.8438 - loss: 0.5536
Validation Loss: 0.5536
Validation Accuracy: 0.8438
